Для разметки датасета был приготовлен 
список тегов взятый из сайта news.itmo.ru для категорий спорт и карьера.

На основе подготовленных тегов был произведен тегов похожих по тематике.
Для поиска похожих тегов были использованны предобученные вектора
http://vectors.nlpl.eu/repository/20/182.zip

In [1]:
import json
from vectorizer import Vectorizer
import copy

In [4]:
!wget http: // vectors.nlpl.eu/repository/20/182.zip -P data/

--2021-08-28 17:04:43--  ftp://http/
           => ‘data/.listing’
Resolving http (http)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘http’
//: Scheme missing.
--2021-08-28 17:04:43--  http://vectors.nlpl.eu/repository/20/182.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 637613799 (608M) [application/zip]
Saving to: ‘data/182.zip’

182.zip             100%[===================>] 608,08M  3,80MB/s    in 2m 49s  

2021-08-28 17:07:32 (3,60 MB/s) - ‘data/182.zip’ saved [637613799/637613799]

FINISHED --2021-08-28 17:07:32--
Total wall clock time: 2m 49s
Downloaded: 1 files, 608M in 2m 49s (3,60 MB/s)


In [5]:
import zipfile
with zipfile.ZipFile("data/182.zip", 'r') as archive:
    stream = archive.open('model.bin')
    vectorizer = Vectorizer(stream)

In [18]:
def load_dataset(path):
    with open(path,"r") as f:
        return json.load(f)

In [19]:
categories = ['science',"education",'startups',"university"]

In [20]:
source = load_dataset("data/news_itmo.json")

In [21]:
categories_tags = {
    'science':set(),
    'education':set(),
    'startups':set(),
    'university':set(),
}

categories_tags_copy = {
    'science':set(),
    'education':set(),
    'startups':set(),
    'university':set(),
}

In [22]:
for cat in source:
    true_cat = ''
    for c in categories:
        if cat.startswith(c):
            true_cat = c
            break
    for post in source[cat].values():
        categories_tags[true_cat].update(post['tags'])
        categories_tags_copy[true_cat].update(post['tags'])

prepared_tags.json ссылка на файл
https://drive.google.com/file/d/1LGiWgOytH4DuU1EWL9r1YyKitsbbCf0o/view?usp=sharing

In [23]:
prepared_categories = load_dataset("data/prepared_tags.json")

In [24]:
prepared_categories_copy = load_dataset("data/prepared_tags.json")

In [25]:
for cat,tags in prepared_categories.items():
    for cat1,tags1 in categories_tags.items():
        tags = set(vectorizer.get_similar_tags(tags,list(tags1)))
        categories_tags_copy[cat1].difference_update(tags)
        prepared_categories_copy[cat] = set(prepared_categories_copy[cat]).union(tags)

Razmetka

In [26]:
sport_career = {
    "sport":dict(),
    "career":dict()
}

In [27]:
for cat,tags in prepared_categories_copy.items():
    source_copy = copy.deepcopy(source)
    for cat1,posts in source.items():
        for post in posts:
            if tags.intersection(set(source[cat1][post]['tags'])):
                sport_career[cat][source[cat1][post]['url']] = source[cat1][post]
                source_copy[cat1].pop(post)
    source = source_copy

In [28]:
other_categories = {
    "science":dict(),
    "education":dict(),
    "startups":dict(),
    "university":dict()
}

In [29]:
for cat,posts in source.items():
    true_cat = ''
    for i in categories:
        if cat.startswith(i):
            true_cat = i
    for post in posts:
        other_categories[true_cat][source[cat][post]['url']] = source[cat][post]

In [30]:
for cat,posts in sport_career.items():
    print(cat,len(posts))

sport 140
career 332


In [33]:
for cat,posts in other_categories.items():
    print(cat,len(posts))

science 686
education 1027
startups 543
university 690


final_dataset.json  ссылка на файл
https://drive.google.com/file/d/12At-458dhTz0RJfcIu9qITnYu--akFKc/view?usp=sharing

In [39]:
with open("data/final_dataset.json","w") as f:
    json.dump({**sport_career,**other_categories},f,indent=2,ensure_ascii=True)